<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.3:
# *Database Programming with Python and SQLite*

In the first part of the lab you will use install and use SQLite, a database program with which we can create and query small databases. The Python library `sqlite3` will be used to connect the database to this Jupyter notebook.

In the second part of the lab, two dataframes will be saved as tables in the SQLite database. Then a number of questions about the data will be asked to practise SQL syntax.

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and database maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands".

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [2]:
import sqlite3
sqlite_db = 'test_db.sqlite' # create a database
conn = sqlite3.connect(sqlite_db) # connect to database
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [5]:
c.execute('DROP TABLE houses;') # Drop the table 'houses'

# Save (commit) the changes:
conn.commit()

In [8]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);') # create new table
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [11]:
# A
last_sale = (None, 4000, 5, 22, 619000) # create a new record call 'last_sale'

# Execute Command
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)',last_sale) # insert the record into the table 'houses'

# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key.

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [14]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
] # create 3 records 'recent_sales'

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
#
c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales) # insert recent_sales into the 'houses' table

conn.commit()

Select all rows from houses

In [17]:
# A
query = 'Select * from houses' # Select all rows from 'houses' table
c.execute(query)
c.fetchall()
# pd.DataFrame(c.fetchall())

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000)]

In [19]:
# Convert to Dataframe
import pandas as pd
query = 'Select * from houses'
c.execute(query)
df = pd.DataFrame(c.fetchall())
column_names = [description[0] for description in c.description]
df.columns = column_names
print(df)

   field1  sqft  bdrms  age   price
0       1  4000      5   22  619000
1       2  2390      4   34  319000
2       3  1870      3   14  289000
3       4  1505      3   90  269000





#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more.

In [22]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('housing-data.csv', dtype = 'i8',
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [25]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [28]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [31]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [34]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [38]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do? It deleted records 52 and 53 from house table

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [43]:
# ANSWER
a = c.execute('SELECT * FROM houses WHERE bdrms == 4')
a.fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

**2. The following query calculates the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [46]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1827.5555555555557, 322892.51851851854),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [49]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database.

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [53]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [57]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

47

Run a query to get the average price of each house size from this table:

In [60]:
#ANSWER
#Connect to the database houses_pandas.
import sqlite3
house = 'house_pandas.sqlite'
conn = sqlite3.connect(house)
c = conn.cursor()
#Perform the query to get average price of each house size 
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)
results = c.execute("SELECT bdrms, AVG(price) AS avg_price FROM houses_pandas GROUP BY bdrms")
results.fetchall()

[(1, 169900.0),
 (2, 280866.6666666667),
 (3, 326403.92),
 (4, 377449.78571428574),
 (5, 699900.0)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database? Pandas is for cleaning the data/transform and push to SQL

2. When would you want to use Pandas on a dataset retrieved from a database ? Pull the pandas for EDA, visualization and machine learning

<a name="guided-practice"></a>


# Reference: SQL Syntax


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate functions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [67]:
# Reading CSV to Dataframe
orders = pd.read_csv('P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('P12-OrderBreakdown.csv', encoding = 'utf-8')

In [68]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [71]:
orders_break_down.head()

,Order ID,Product Name,Discount,Unit Price,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [74]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [76]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes

In [79]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [81]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
unit_price      float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [84]:
# Establishing Local DB connection
db_connection = sqlite3.connect('eshop.db.sqlite')
c = db_connection.cursor()
# Save these two dataframes as tables in sqlite
orders.to_sql('orders', con = db_connection, if_exists = 'replace', index = False)
orders_break_down.to_sql('orders_break_down', con = db_connection, if_exists = 'replace', index = False)
db_connection.commit()

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [87]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT 
            *
        FROM 
            orders
        LIMIT 
            10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


#### 5. Find the number of orders for each customer

In [90]:
# Find the number of DISTINCT order ids for each customer name
query = """
        SELECT 
            customer_name as Customer , 
            COUNT(DISTINCT order_id) as Distinct_Order
        FROM 
            orders
        GROUP BY 
            customer_name
        ORDER BY
            Distinct_Order DESC;
        """

orders_distinct = sql.read_sql(query, con=db_connection)
orders_distinct

,Customer,Distinct_Order
0,John Grady,13
1,Joel Jenkins,12
2,Aaron Smayling,12
3,Yoseph Carroll,11
4,Sarah Brown,11
...,...,...
787,Brooke Gillingham,1
788,Barbara Fisher,1
789,Art Ferguson,1
790,Andy Reiter,1


#### 6. Find the number of customers for each country

In [93]:
# Find the number of customers for each country
query = """
        SELECT 
            country as Country, 
            COUNT(customer_name) as No_Customer
        FROM 
            orders
        GROUP BY 
            country;
        """

orders_customer = sql.read_sql(query, con=db_connection)
orders_customer

,Country,No_Customer
0,Austria,135
1,Belgium,68
2,Denmark,29
3,Finland,34
4,France,991
5,Germany,806
6,Ireland,50
7,Italy,493
8,Netherlands,194
9,Norway,37


#### 7.A Select discount, unit price, quantity for each order from orders_break_down table

In [96]:
#  Select discount, unit price, quantity for each order from orders_break_down table
query = """
        SELECT 
            order_id,
            discount, 
            unit_price, 
            quantity
        FROM 
            orders_break_down;
        """

orders_breakdown = sql.read_sql(query, con=db_connection)
orders_breakdown

,order_id,discount,unit_price,quantity
0,IT-2011-3647632,0.5,45.0,3
1,ES-2011-4869686,0.0,854.0,7
2,ES-2011-4939443,0.0,140.0,3
3,IT-2011-2942451,0.5,27.0,2
4,IT-2011-2942451,0.5,17.0,2
...,...,...,...,...
8042,ES-2014-3458802,0.0,245.0,2
8043,IT-2014-3715679,0.5,30.0,2
8044,IT-2014-3715679,0.5,23.0,4
8045,IT-2014-3715679,0.5,108.0,3


#### 7.B Select discount, unit price, quantity, discounted price for each order from orders_break_down table

> discounted price = unit_price * quantity * (1 - discount)

In [99]:
# Select discount, unit price, quantity, discounted price for each order from orders_break_down table with discounted price in descending order
query = """
        SELECT 
            order_id,
            discount, 
            unit_price, 
            quantity, 
            unit_price*quantity*(1-discount) AS discounted_price
        FROM 
            orders_break_down
        ORDER BY 
            discounted_price DESC;
        """

orders_discounted_price = sql.read_sql(query, con=db_connection)
orders_discounted_price

,order_id,discount,unit_price,quantity,discounted_price
0,ES-2011-3248922,0.0,6517.0,12,78204.0
1,ES-2014-3785216,0.1,5729.0,14,72185.4
2,ES-2012-2183106,0.1,4748.0,14,59824.8
3,ES-2014-2637201,0.0,4141.0,13,53833.0
4,ES-2013-2860574,0.0,5274.0,10,52740.0
...,...,...,...,...,...
8042,ES-2012-4572658,0.5,5.0,1,2.5
8043,IT-2013-5843979,0.5,5.0,1,2.5
8044,IT-2011-1320928,0.5,4.0,1,2.0
8045,IT-2014-3944375,0.5,4.0,1,2.0


#### 7.C Select all columns and also the discounted price from orders_break_down table where discounted price (defined above) is greater than 100

In [102]:
# Select all orders and also the discounted price from orders_break_down table where discounted price (defined above) is greater than 100,
# ordered by discounted price in ascending order
query = """
        SELECT 
            discount, 
            unit_price, 
            quantity, 
            unit_price*quantity*(1-discount) AS discounted_price
        FROM 
            orders_break_down
        WHERE 
            discounted_price > 100
        ORDER BY 
            discounted_price;
        """

orders_discounted_price_100 = sql.read_sql(query, con=db_connection)
orders_discounted_price_100

,discount,unit_price,quantity,discounted_price
0,0.5,67.0,3,100.5
1,0.5,67.0,3,100.5
2,0.1,56.0,2,100.8
3,0.4,42.0,4,100.8
4,0.4,42.0,4,100.8
...,...,...,...,...
6245,0.0,5274.0,10,52740.0
6246,0.0,4141.0,13,53833.0
6247,0.1,4748.0,14,59824.8
6248,0.1,5729.0,14,72185.4


#### 8. Select all order ids, customers and the product they have bought

In [105]:
# Select all order_ids, customer names and the products (product_name) they have bought
query = """
        SELECT 
            a.order_id, 
            a.customer_name, 
            b.product_name
        FROM 
            orders a
        INNER JOIN 
            orders_break_down b ON a.order_id = b.order_id;
        """

orders_join = sql.read_sql(query, con=db_connection)
orders_join

,order_id,customer_name,product_name
0,IT-2011-3647632,Eugene Moren,"Enermax Note Cards, Premium"
1,ES-2011-4869686,Dorothy Dickinson,"Dania Corner Shelving, Traditional"
2,ES-2011-4939443,Arthur Prichep,"Binney & Smith Sketch Pad, Easy-Erase"
3,IT-2011-2942451,Grant Thornton,"Boston Markers, Easy-Erase"
4,IT-2011-2942451,Grant Thornton,"Eldon Folders, Single Width"
...,...,...,...
8042,ES-2014-3638865,Susan Vittorini,"Fellowes Shelving, Single Width"
8043,ES-2014-4785777,Dennis Pardue,"Wilson Jones Index Tab, Economy"
8044,IT-2014-3715679,Jim Kriz,"Avery Binder Covers, Recycled"
8045,IT-2014-3715679,Jim Kriz,"BIC Pencil Sharpener, Fluorescent"


#### 9.A Find the number of 'Furniture' orders for each country

In [108]:
# Find the number of 'Furniture' orders for each country
query = """
        SELECT 
            a.country AS Country, 
            COUNT (a.order_id) AS No_furniture
        FROM 
            orders a
        INNER JOIN 
            orders_break_down b ON a.order_id = b.order_id
        WHERE 
            b.category = "Furniture"
        GROUP BY 
            a.country;
        """

orders_furniture = sql.read_sql(query, con=db_connection)
orders_furniture

,Country,No_furniture
0,Austria,40
1,Belgium,20
2,Denmark,9
3,Finland,16
4,France,299
5,Germany,264
6,Ireland,12
7,Italy,136
8,Netherlands,76
9,Norway,6


#### 9.B Select 'Furniture' orders for the country Denmark

In [111]:
# Select all columns from the orders table which are 'Furniture' orders for the country Denmark
query = """
        SELECT 
            *
        FROM 
            orders a
        INNER JOIN 
            orders_break_down b ON a.order_id = b.order_id
        WHERE 
            b.category = "Furniture" AND a.country = "Denmark";
        """

orders_furniture_denmark = sql.read_sql(query, con=db_connection)
orders_furniture_denmark

,order_id,order_date,customer_name,country,order_id,product_name,discount,unit_price,quantity,category
0,ES-2011-1586186,2011-06-16,Vivek Grady,Denmark,ES-2011-1586186,"Office Star Bag Chairs, Set of Two",0.5,87.0,3,Furniture
1,IT-2012-4084224,2012-02-15,Mick Brown,Denmark,IT-2012-4084224,"Rubbermaid Door Stop, Ergonomic",0.6,17.0,1,Furniture
2,IT-2013-4649004,2013-05-31,Bart Folk,Denmark,IT-2013-4649004,"Novimex Executive Leather Armchair, Red",0.5,920.0,4,Furniture
3,ES-2013-3422294,2013-09-07,Dave Brooks,Denmark,ES-2013-3422294,"Advantus Door Stop, Duo Pack",0.6,36.0,2,Furniture
4,ES-2014-3653593,2014-02-10,Ivan Liston,Denmark,ES-2014-3653593,"Deflect-O Clock, Black",0.6,161.0,8,Furniture
5,ES-2014-3653593,2014-02-10,Ivan Liston,Denmark,ES-2014-3653593,"Harbour Creations Bag Chairs, Set of Two",0.5,32.0,1,Furniture
6,ES-2014-1415600,2014-07-07,Damala Kotsonis,Denmark,ES-2014-1415600,"Deflect-O Door Stop, Duo Pack",0.6,70.0,4,Furniture
7,IT-2014-1978058,2014-11-13,Anthony Jacobs,Denmark,IT-2014-1978058,"Harbour Creations Steel Folding Chair, Red",0.5,98.0,2,Furniture
8,ES-2014-5784412,2014-11-25,Pete Kriz,Denmark,ES-2014-5784412,"Eldon Light Bulb, Black",0.6,50.0,5,Furniture


#### 10. Find the total sales with discount and without discount for each country

In [114]:
# Find the total sales with discount (discount > 0) and without discount (discount = 0) for each country
# total sales with discount = sum (unit price * quantity * (1 - discount)) for each country where we only sum over rows where discount > 0
# total sales without discount = sum (unit price * quantity) for each country where we only sum over rows where discount = 0
# hint: use case statements
query = """
        SELECT 
            a.country,
            SUM(CASE WHEN b.discount > 0 THEN b.unit_price*b.quantity*(1-b.discount) ELSE 0 END) AS Total_Sales_with_Discount,
            SUM(CASE WHEN b.discount = 0 THEN b.unit_price*b.quantity ELSE 0 END) AS Total_Sales_no_Discount
        FROM 
            orders a
        INNER JOIN 
            orders_break_down b ON a.order_id = b.order_id
        GROUP BY 
            a.country;
        """

orders_discount_total = sql.read_sql(query, con=db_connection)
orders_discount_total

,country,Total_Sales_with_Discount,Total_Sales_no_Discount
0,Austria,0.0,391694.0
1,Belgium,0.0,229829.0
2,Denmark,17117.8,0.0
3,Finland,0.0,82001.0
4,France,2063927.0,665914.0
5,Germany,1026641.8,1206817.0
6,Ireland,37783.6,0.0
7,Italy,214253.0,806777.0
8,Netherlands,195750.9,0.0
9,Norway,0.0,99878.0


#### 11.A Find the total quantity, total sales for each country

In [116]:
# Select sum of quantities, total sales (sum (unit_price * quantity * (1 - discount))) for each country
query = """
        SELECT 
            a.country AS Country,
            SUM(b.quantity) AS Total_Qty, 
            SUM(b.unit_price*b.quantity*(1-b.discount)) AS Total_Sales
        FROM 
            orders a
        INNER JOIN 
            orders_break_down b ON a.order_id = b.order_id
        GROUP BY 
            a.country;
        """

orders_qty_sales = sql.read_sql(query, con=db_connection)
orders_qty_sales

,Country,Total_Qty,Total_Sales
0,Austria,973,391694.0
1,Belgium,532,229829.0
2,Denmark,204,17117.8
3,Finland,201,82001.0
4,France,7329,2729841.0
5,Germany,6179,2233458.8
6,Ireland,392,37783.6
7,Italy,3612,1021030.0
8,Netherlands,1526,195750.9
9,Norway,261,99878.0


#### 11.B Based on total sales for each country, find the top 3 countries and their quantities

In [120]:
# Based on total sales for each country (sum (unit_price * quantity * (1 - discount))), find the top 3 countries and their quantities
query = """
        SELECT 
            a.country AS Country,
            SUM(b.quantity) AS Total_Qty, 
            SUM(b.unit_price*b.quantity*(1-b.discount)) AS Total_Sales
        FROM 
            orders a
        INNER JOIN 
            orders_break_down b ON a.order_id = b.order_id
        GROUP BY 
            a.country
        ORDER BY 
            Total_Sales DESC
            LIMIT 3;
        """

orders_sales_top3 = sql.read_sql(query, con=db_connection)
orders_sales_top3

,Country,Total_Qty,Total_Sales
0,France,7329,2729841.0
1,Germany,6179,2233458.8
2,United Kingdom,4917,2078715.7


#### 11.C Based on total quantities for each country, find the bottom 3 countries, their total sales and total quantities

In [123]:
# Based on total quantities for each country, find the bottom 3 countries, their total sales (sum(unit_price * quantity * (1 - discount))) and total quantities
query = """
        SELECT 
            a.country AS Country,
            SUM(b.quantity) AS Total_Qty, 
            SUM(b.unit_price*b.quantity*(1-b.discount)) AS Total_Sales
        FROM 
            orders a
        INNER JOIN 
            orders_break_down b ON a.order_id = b.order_id
        GROUP BY 
            a.country
        ORDER BY 
            Total_Qty ASC
            LIMIT 3;
        """

orders_qty_btm3 = sql.read_sql(query, con=db_connection)
orders_qty_btm3

,Country,Total_Qty,Total_Sales
0,Finland,201,82001.0
1,Denmark,204,17117.8
2,Norway,261,99878.0


#### 12. Find average sales by category for the country 'France'

In [130]:
# Find average sales (i.e. avg( unit_price * quantity * (1 - discount) ) ) by category for the country 'France' in descending order
query = """
        SELECT 
            b.category AS Category,
            AVG(b.unit_price*b.quantity*(1-b.discount)) AS Avg_Sales
        FROM 
            orders a
        INNER JOIN 
            orders_break_down b ON a.order_id = b.order_id
        WHERE 
            a.country = 'France'
        GROUP BY 
            b.category
        ORDER BY 
            Avg_Sales Desc
        """

orders_france_category = sql.read_sql(query, con=db_connection)
orders_france_category

,Category,Avg_Sales
0,Furniture,2632.707860
1,Technology,2508.548575
2,Office Supplies,791.520390


#### 13. Find the country, category and average sales where average sales is the highest

In [136]:
# Find the country, category and average sales (i.e. avg( unit_price * quantity * (1 - discount) ) ) where average sales is the highest
query = """
        SELECT 
            b.category AS Category,
            a.country AS country,
            AVG(b.unit_price*b.quantity*(1-b.discount)) AS Avg_Sales
        FROM 
            orders a
        INNER JOIN 
            orders_break_down b ON a.order_id = b.order_id
        GROUP BY 
            a.country,
            b.category
        ORDER BY 
            Avg_Sales DESC
            LIMIT 1;
        """

orders_avgsales_max = sql.read_sql(query, con=db_connection)
orders_avgsales_max

,Category,country,Avg_Sales
0,Technology,Switzerland,6487.846154


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams.
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  



---



---



> > > > > > > > > © 2025 Institute of Data


---



---



